- Update config.yml
- Update params.yml
- Update entity
- Update the configuration manager in src config
- Update the components
- Update the pipeline
- Update main.py
- Update app.py

In [1]:
import os

In [2]:
%pwd

'd:\\Project\\Text-Summarizer\\textSummarizer\\src\\textSummarizer\\notebooks'

In [3]:
os.chdir("../../../")

In [4]:
%pwd

'd:\\Project\\Text-Summarizer\\textSummarizer'

In [5]:
# Update entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [8]:
read_yaml(CONFIG_FILE_PATH).artifact_root

[2025-03-30 23:24:49,903 - INFO - (common)]: YAML file config\config.yaml loaded successfully.


'artifacts'

In [9]:
PARAMS_FILE_PATH

WindowsPath('parameters.yaml')

In [10]:
# Update the configuration manager in src config
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [11]:
# Update the components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}.")
            logger.info(f"File size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        print(f"Unzipping file to {unzip_path}")
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    
    

In [13]:
# Update the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-30 23:24:49,980 - INFO - (common)]: YAML file config\config.yaml loaded successfully.
[2025-03-30 23:24:49,981 - INFO - (common)]: YAML file parameters.yaml loaded successfully.
[2025-03-30 23:24:49,982 - INFO - (common)]: Directory artifacts created or already exists.
[2025-03-30 23:24:49,983 - INFO - (common)]: Directory artifacts/data_ingestion created or already exists.
[2025-03-30 23:24:51,233 - INFO - (3485655966)]: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3B60:3324FC:70FBFC:8D70FC:67EA0B02
Accept-Ranges: bytes
Date: Mon, 31 Mar 2025 03:24:51 GMT
Via